## Unsupervised ML variant pathogenicity prediction

Part 1 - Common Mutations Between Datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import norm

In [2]:
# read files
result_df = pd.read_csv("result_df.csv")
Final_table_unfiltered= pd.read_csv("Final_table_unfiltered.csv")

In [4]:
dataf = Final_table_unfiltered.drop("#Uploaded_variation", axis=1)
dataf = dataf.drop("Protein_position", axis=1)
dataf

,POS,Datasets,Healthy,SLE,Occurrence,REF,ALT,GQ,QUAL,DP,...,FILTER,IMPACT,BIOTYPE,Consequence,AF,Amino_acids,CADD_PHRED,SIFT,PolyPhen,score
0,68003029,2,0,3,3,G,T,99.000000,875.306667,65.000000,...,PASS,MODERATE,"protein_coding,nonsense_mediated_decay,retaine...","missense_variant,missense_variant,NMD_transcri...",0.0028,L129I,26.000,0.02,0.951,85.855719
1,67999234,2,0,3,3,G,A,99.000000,1490.640000,72.000000,...,PASS,MODERATE,"protein_coding,nonsense_mediated_decay,retaine...","missense_variant,3_prime_UTR_variant,NMD_trans...",0.0028,P209L,25.400,0.00,0.996,71.711330
2,67991568,2,0,1,1,C,T,99.000000,968.640000,41.000000,...,PASS,MODERATE,"protein_coding,retained_intron","missense_variant,downstream_gene_variant",0.0024,G591E,17.490,0.48,0.995,64.518414
3,67999680,2,0,1,1,C,A,50.000000,42.640000,50.000000,...,PASS,MODERATE,"protein_coding,nonsense_mediated_decay,retaine...","missense_variant,splice_region_variant,missens...",0.0000,R131S,25.000,0.00,0.350,57.554599
4,67997719,3,0,1,1,C,A,99.000000,0.000000,346.000000,...,PASS,MODERATE,"protein_coding,nonsense_mediated_decay,retaine...","missense_variant,downstream_gene_variant,upstr...",0.0000,V288L,26.500,0.05,0.469,51.914286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,67995404,"1,2,3",2,19,21,T,C,33.857143,64.594762,4.428571,...,PASS,MODIFIER,"protein_coding,nonsense_mediated_decay,retaine...","intron_variant,downstream_gene_variant,intron_...",0.2324,-,5.551,0.00,0.000,4.298182
659,67994228,"1,3",13,22,35,A,C,20.914286,84.730000,3.457143,...,PASS,MODIFIER,"protein_coding,nonsense_mediated_decay,retaine...","intron_variant,downstream_gene_variant,intron_...",0.5238,-,1.069,0.00,0.000,4.231546
660,68000750,"2,3",1,3,4,T,G,35.000000,78.725000,5.000000,...,PASS,MODIFIER,"protein_coding,nonsense_mediated_decay,retaine...","intron_variant,intron_variant,NMD_transcript_v...",0.0148,-,0.154,0.00,0.000,4.230818
661,67999031,"1,2,3",3,3,6,T,G,26.000000,68.673333,2.833333,...,PASS,MODIFIER,"protein_coding,nonsense_mediated_decay,retaine...","intron_variant,intron_variant,NMD_transcript_v...",0.1595,-,3.338,0.00,0.000,4.164858


In [6]:
# Define weights
# Original weights dictionary remains the same
weights = {
    'intergenic_variant': 1,
    'feature_truncation': 3,
    'regulatory_region_variant': 3,
    'feature_elongation': 3,
    'regulatory_region_amplification': 3,
    'regulatory_region_ablation': 3,
    'TF_binding_site_variant': 3,
    'TFBS_amplification': 3,
    'TFBS_ablation': 3,
    'downstream_gene_variant': 3,
    'upstream_gene_variant': 3,
    'non_coding_transcript_variant': 3,
    'NMD_transcript_variant': 3,
    'intron_variant': 3,
    'non_coding_transcript_exon_variant': 3,
    '3_prime_UTR_variant': 5,
    '5_prime_UTR_variant': 5,
    'mature_miRNA_variant': 5,
    'coding_sequence_variant': 5,
    'synonymous_variant': 5,
    'stop_retained_variant': 5,
    'incomplete_terminal_codon_variant': 5,
    'splice_region_variant': 5,
    'protein_altering_variant': 10,
    'missense_variant': 10,
    'inframe_deletion': 15,
    'inframe_insertion': 15,
    'transcript_amplification': 15,
    'start_lost': 15,
    'stop_lost': 15,
    'frameshift_variant': 20,
    'stop_gained': 20,
    'splice_donor_variant': 20,
    'splice_acceptor_variant': 20,
    'transcript_ablation': 20
}

# Helper function for dataset count that was missing
def get_dataset_count(dataset_str):
    """Count the number of datasets a variant appears in"""
    if pd.isna(dataset_str) or dataset_str == '':
        return 0
    return len(str(dataset_str).split(','))


Top contributing features to variant scoring:
  - CADD: 0.338
  - Base_consequence: 0.331
  - Pathogenicity_composite: 0.330
  - AA_change_factor: 0.329
  - Impact: 0.326

ML Score Distribution:
Very high risk (85-100): 113 variants (17.0%)
High risk (70-84): 158 variants (23.8%)
Moderate-high risk (55-69): 75 variants (11.3%)
Moderate risk (40-54): 151 variants (22.8%)
Low-moderate risk (25-39): 99 variants (14.9%)
Low risk (10-24): 39 variants (5.9%)
Very low risk (0-9): 28 variants (4.2%)


In [ ]:
# Function to calculate the score
def ml_sle_variant_scoring(df):
    """
     version of SLE variant scoring with improved feature extraction,
    better handling of missing data, and ensemble anomaly detection.
    """
    # Initialize features list
    features = []
    missing_data_flags = []  # Track which variants have missing prediction data
    feature_names = []  # Track feature names for interpretability
    
    for _, row in df.iterrows():
        # Track missing prediction data - expanded to track partial missing data
        missing_polyphen = pd.isna(row['PolyPhen'])
        missing_sift = pd.isna(row['SIFT'])
        missing_cadd = pd.isna(row['CADD_PHRED'])
        missing_clinpred = pd.isna(row['ClinPred'])
        
        # Count how many prediction tools are missing
        missing_count = sum([missing_polyphen, missing_sift, missing_cadd, missing_clinpred])
        missing_data_flags.append(missing_count / 4.0)  # Normalized missing data ratio
        
        # Split consequences into list and get unique values
        consequences = list(set(str(row['Consequence']).split(',')))
        
        # Get base score (maximum weight among consequences)
        base_score = max(weights.get(consequence.strip(), 0) for consequence in consequences)
        
        #  Consider the top 3 consequences instead of just the maximum
        sorted_weights = sorted([weights.get(consequence.strip(), 0) for consequence in consequences], 
                              reverse=True)
        top_consequences_score = sum(sorted_weights[:min(3, len(sorted_weights))]) / 3
        
        # Functional impact scores with improved defaults and handling
        # PolyPhen - higher is more deleterious
        if pd.notna(row['PolyPhen']):
            polyphen_score = max(0.1, float(row['PolyPhen']))  # Ensure non-zero values
        else:
            polyphen_score = 0.4  # Moderate default
        
        # SIFT - lower is more deleterious, so invert
        if pd.notna(row['SIFT']):
            sift_score = max(0.001, float(row['SIFT']))  # Ensure non-zero values
            sift_score = 1 - sift_score  # Invert so higher is more deleterious
        else:
            sift_score = 0.6  # Moderate default after inversion
            
        # ClinPred - higher is more pathogenic
        if pd.notna(row['ClinPred']):
            clinpred = max(0.1, float(row['ClinPred']))  # Ensure non-zero values
        else:
            clinpred = 0.5  # Moderate default

        # CADD
        cadd_phred = float(row['CADD_PHRED']) if pd.notna(row['CADD_PHRED']) else 0
        if cadd_phred >= 20:
            base_score += 10
        elif cadd_phred >= 10:
            base_score += 5
        # CADD - higher is more deleterious
        if pd.notna(row['CADD_PHRED']):
            cadd = max(1.0, float(row['CADD_PHRED']))  # Ensure non-zero values
        else:
            cadd = 15  # Moderate default
        
        #  Create a composite pathogenicity score
        normalized_cadd = min(1, cadd / 35)  # Normalize CADD (increased max from 30 to 35)
       """ # Give CADD double weight compared to others
        cadd_weight = 0.4  # 40% weight to CADD
        other_weight = 0.2  # 20% weight each to other predictors (total 60%)
        
        pathogenicity_composite = (
            normalized_cadd * cadd_weight +
            polyphen_score * other_weight + 
            sift_score * other_weight + 
            clinpred * other_weight
        )"""
       pathogenicity_composite = (polyphen_score + sift_score + clinpred + normalized_cadd) / 4
        
        # SLE-specific features - critical for disease association
        healthy_presence = float(row['Healthy']) if pd.notna(row['Healthy']) else 0
        sle_presence = float(row['SLE']) if pd.notna(row['SLE']) else 0
        
        #  More nuanced ratio calculation
        if healthy_presence == 0:
            if sle_presence > 0:
                sle_healthy_ratio = 10  # SLE-only variants
            else:
                sle_healthy_ratio = 0   # No presence in either group
        else:
            # Log-scaled ratio to better handle outliers
            ratio = sle_presence / (healthy_presence + 0.001)
            sle_healthy_ratio = min(10, np.log1p(ratio) * 3)
            
        #  Disease enrichment score
        disease_enrichment = np.log1p(sle_presence) - np.log1p(healthy_presence)
        
        # Frequency metrics with improved normalization
        dataset_coverage = get_dataset_count(row['Datasets'])
        normalized_dataset_coverage = min(1.0, dataset_coverage / 5)  # Cap at 5 datasets
        
        max_occurrence = df['Occurrence'].max() if 'Occurrence' in df.columns else 1
        normalized_occurrence = float(row['Occurrence']) / max_occurrence if max_occurrence > 0 else 0
        
        #  Rarity score with log-scaling for better sensitivity
        af = float(row['AF']) if pd.notna(row['AF']) else 0
        if af == 0:
            rarity_score = 1.0  # Maximum rarity for zero frequency
        else:
            # Log-transformed AF with stronger penalty for common variants
            rarity_score = max(0, 1 - (np.log1p(af * 1000) / np.log1p(1000)))
            # Extra boost for very rare variants
            if af < 0.001:
                rarity_score *= 1.5
        
        # Impact score with confidence weighting
        impact_score = 0.5  # Default for LOW or missing
        if pd.notna(row['IMPACT']):
            if row['IMPACT'] == 'HIGH':
                impact_score = 1.8  # Increased weight
            elif row['IMPACT'] == 'MODERATE':
                impact_score = 1.2
        
        # Confidence score
        confidence = float(row['confidence']) if pd.notna(row['confidence']) else 0
        
        #  Check for protein structural changes
        aa_change = False
        severe_aa_change = False
        
        if 'Amino_acids' in row and pd.notna(row['Amino_acids']):
            aa_str = str(row['Amino_acids'])
            if (aa_str != "0" and aa_str.strip() != "" and not aa_str.endswith("=") and aa_str != "-"):
                aa_change = True
                
                # Check for chemically significant amino acid changes if format is like "A/B"
                if '/' in aa_str:
                    aa_parts = aa_str.split('/')
                    if len(aa_parts) == 2:
                        # Define amino acid property groups
                        hydrophobic = set(['A', 'I', 'L', 'M', 'F', 'V', 'P', 'G'])
                        polar = set(['Q', 'N', 'H', 'S', 'T', 'Y', 'C', 'W'])
                        charged_positive = set(['K', 'R'])
                        charged_negative = set(['D', 'E'])
                        
                        def get_aa_group(aa):
                            if aa in hydrophobic: return 'hydrophobic'
                            if aa in polar: return 'polar'
                            if aa in charged_positive: return 'positive'
                            if aa in charged_negative: return 'negative'
                            return 'unknown'
                        
                        orig = aa_parts[0][0] if len(aa_parts[0]) > 0 else ''
                        new = aa_parts[1][0] if len(aa_parts[1]) > 0 else ''
                        
                        if orig and new and orig != new:
                            orig_group = get_aa_group(orig)
                            new_group = get_aa_group(new)
                            
                            # Changes between property groups are more severe
                            if orig_group != new_group and orig_group != 'unknown' and new_group != 'unknown':
                                severe_aa_change = True
        
        # Apply amino acid change factor
        aa_factor = 2.5 if severe_aa_change else (1.8 if aa_change else 1.0)
        
        #  Consider functional importance with more nuance
        functional_impact = (polyphen_score >= 0.447 or sift_score >= 0.95 or normalized_cadd >= 0.7 or clinpred >= 0.7)
        likely_functional = (polyphen_score >= 0.2 or sift_score >= 0.6 or normalized_cadd >= 0.5 or clinpred >= 0.5)
        
        function_score = 2.0 if functional_impact else (1.5 if likely_functional else 1.0)
        
        #  Create more nuanced feature vector
        variant_features = [
            sle_presence * 2.5,                        # Increased SLE presence weight
            -healthy_presence * 1.2,                   # Increased penalty for healthy presence
            sle_healthy_ratio * 0.4,                   # SLE/healthy ratio
            disease_enrichment * 1.5,                  # Disease enrichment score
            normalized_occurrence * 0.8,               # Moderated occurrence weight
            normalized_dataset_coverage * 0.7,         # Dataset coverage
            rarity_score * 1.5,                        # Rarity importance
            confidence * 0.8,                          # Confidence score
            clinpred * 1.2 * aa_factor,                # ClinPred boosted by aa changes
            polyphen_score * 1.2 * aa_factor,          # PolyPhen boosted by aa changes
            sift_score * 1.2 * aa_factor,              # SIFT boosted by aa changes
            normalized_cadd * 1.2 * aa_factor,         # CADD boosted by aa changes
            pathogenicity_composite * 1.8,             # Composite pathogenicity
            impact_score * 1.5,                        # Increased impact weight
            aa_factor * 1.2,                           # Amino acid change factor
            function_score * 1.5,                      # Functional importance
            base_score * 0.4,                          # Base consequence score
            top_consequences_score * 0.5,              # Top consequences average
        ]
        
        # Track feature names for interpretability
        if len(feature_names) == 0:
            feature_names = [
                "SLE_presence", "Healthy_presence_penalty", "SLE_healthy_ratio", 
                "Disease_enrichment", "Occurrence", "Dataset_coverage", 
                "Rarity", "Confidence", "ClinPred", "PolyPhen", 
                "SIFT", "CADD", "Pathogenicity_composite", "Impact", 
                "AA_change_factor", "Function_score", "Base_consequence", 
                "Top_consequences_avg"
            ]
        
        features.append(variant_features)
    
    # Convert to numpy array
    features = np.array(features)
    missing_data_flags = np.array(missing_data_flags)
    
    #  Scale features with robust scaling for better outlier handling
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    #  Apply PCA with variance explanation analysis
    pca = PCA()
    features_pca = pca.fit_transform(features_scaled)
    
    # Determine optimal number of components (explaining 95% variance)
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    
    # Reduce to optimal components
    pca = PCA(n_components=optimal_components)
    features_pca = pca.fit_transform(features_scaled)
    
    #  Ensemble of anomaly detection methods
    # 1. Isolation Forest with different contamination levels
    iforest_loose = IsolationForest(n_estimators=100, contamination=0.2, random_state=42)
    iforest_tight = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)
    
    iforest_loose.fit(features_pca)
    iforest_tight.fit(features_pca)
    
    iforest_loose_scores = -iforest_loose.score_samples(features_pca)
    iforest_tight_scores = -iforest_tight.score_samples(features_pca)
    
    # 2. Local Outlier Factor with different neighborhood sizes
    lof_small = LocalOutlierFactor(n_neighbors=10, contamination=0.15, novelty=True)
    lof_large = LocalOutlierFactor(n_neighbors=20, contamination=0.15, novelty=True)
    
    lof_small.fit(features_pca)
    lof_large.fit(features_pca)
    
    lof_small_scores = -lof_small.score_samples(features_pca)
    lof_large_scores = -lof_large.score_samples(features_pca)
    
    #  Ensemble scoring with weighted combination
    ensemble_scores = (
        0.3 * iforest_loose_scores + 
        0.2 * iforest_tight_scores + 
        0.3 * lof_small_scores + 
        0.2 * lof_large_scores
    )
    
    #  Add distance-based metrics in PCA space
    # Distance from origin in PCA space (useful for finding extreme values)
    pca_distances = np.sqrt(np.sum(features_pca**2, axis=1))
    pca_distances = (pca_distances - np.min(pca_distances)) / (np.max(pca_distances) - np.min(pca_distances))
    
    #  Create final scoring with adaptive weighting
    # Weight anomaly scores higher for variants with amino acid changes or functional predictions
    aa_present = np.array([features[i, 14] > 1.2 for i in range(len(features))])
    functional_present = np.array([features[i, 15] > 1.2 for i in range(len(features))])
    
    # Adjust weights based on variant characteristics
    ensemble_weight = 0.6 + 0.1 * aa_present + 0.1 * functional_present
    distance_weight = 1.0 - ensemble_weight
    
    # Blend scores with adaptive weights
    ml_scores = ensemble_weight * ensemble_scores + distance_weight * pca_distances
    
    #  Apply sigmoid transformation with adaptive parameters
    # Use steeper sigmoid for variants with more prediction data
    sigmoid_steepness = 2.0 - missing_data_flags * 0.5  # Less steep for variants with missing data
    sigmoid_shift = np.mean(ml_scores) * 0.9  # Shift sigmoid slightly left of mean
    
    # Apply individualized sigmoid to each score
    sigmoid_scores = 1 / (1 + np.exp(-sigmoid_steepness * (ml_scores - sigmoid_shift)))
    
    #  Boost scores for SLE-specific variants and those with missing data
    for i in range(len(features)):
        boost_factor = 1.0
        
        # Variants present in SLE but not or rarely in healthy controls
        if features[i, 0] > 0 and features[i, 1] > -0.5:
            boost_factor *= 1.3  # Increased from 1.25
        
        # Higher boost for variants with strong SLE presence and no healthy presence
        if features[i, 0] > 1.5 and features[i, 1] == 0:
            boost_factor *= 1.2  # Additional multiplier
        
        # Variants with amino acid changes
        if features[i, 14] > 1.5:
            boost_factor *= 1.2  # Increased from 1.15
        
        # Variants with high pathogenicity composite but missing some predictions
        if features[i, 12] > 0.7 and missing_data_flags[i] > 0.25:
            boost_factor *= 1.15  # Increased from 1.1
        
        # CRITICAL:  handling for variants with missing prediction data
        # We now use a more granular approach based on how much data is missing
        # and what other evidence we have
        
        # Partial missing data (25-50%)
        if 0.25 < missing_data_flags[i] <= 0.5:
            # Has disease association or functional evidence
            if features[i, 0] > 0.5 or features[i, 14] > 1.2 or features[i, 16] > 8:
                boost_factor *= 1.15
        
        # Significant missing data (50-75%)
        elif 0.5 < missing_data_flags[i] <= 0.75:
            # Strong disease association or important consequence
            if features[i, 0] > 1.0 or features[i, 3] > 0.5 or features[i, 16] > 10:
                boost_factor *= 1.35
            
            # Has amino acid changes
            if features[i, 14] > 1.2:
                boost_factor *= 1.2
            
            # Rare variant with high impact
            if features[i, 6] > 0.8 and features[i, 13] > 1.0:
                boost_factor *= 1.25
                
            # Missing particular tools but has others
            # If CADD is present but others missing
            if features[i, 11] > 0.7 and missing_data_flags[i] > 0.5:
                boost_factor *= 1.15
            
            # If any predictor shows high pathogenicity despite others missing
            max_pred_score = max(features[i, 8:12])  # Max of prediction scores
            if max_pred_score > 0.8:
                boost_factor *= 1.2
        
        # Mostly or completely missing data (>75%)
        elif missing_data_flags[i] > 0.75:
            # Still has strong disease association
            if features[i, 0] > 1.0 or features[i, 3] > 0.7:
                boost_factor *= 1.45  # Increased from 1.35
            
            # Important consequence type
            if features[i, 16] > 15:  # Higher threshold for more severe consequences
                boost_factor *= 1.5
            elif features[i, 16] > 10:
                boost_factor *= 1.3
            
            # Has amino acid changes
            if features[i, 14] > 1.5:  # Severe AA change
                boost_factor *= 1.4  # Increased from 1.2
            elif features[i, 14] > 1.2:  # Any AA change
                boost_factor *= 1.25
            
            # Rare variant with high impact
            if features[i, 6] > 0.9 and features[i, 13] > 1.5:  # Very rare, high impact
                boost_factor *= 1.5  # Increased from 1.3
            elif features[i, 6] > 0.8 and features[i, 13] > 1.0:  # Rare, moderate impact
                boost_factor *= 1.3
            
            # Strong presence across multiple datasets
            if features[i, 5] > 0.6:  # Present in multiple datasets
                boost_factor *= 1.2
            
            # Extremely high confidence even with missing predictions
            if features[i, 7] > 0.8:  # High confidence
                boost_factor *= 1.25
            
            # If the variant has strong indicators despite missing almost all predictions,
            # give it an additional boost
            if missing_data_flags[i] > 0.9 and (
                (features[i, 0] > 0 and features[i, 1] == 0) or  # SLE-specific
                features[i, 16] > 15 or  # High-impact consequence
                features[i, 14] > 1.5    # Severe AA change
            ):
                boost_factor *= 1.15  # Extra boost for critical variants with minimal data
        
        # Apply combined boost factor
        if boost_factor > 1.0:
            sigmoid_scores[i] = min(1.0, sigmoid_scores[i] * boost_factor)
    
    # Final normalization to 0-100 range
    normalized_scores = (sigmoid_scores - np.min(sigmoid_scores)) / (np.max(sigmoid_scores) - np.min(sigmoid_scores)) * 100
    
    #  Apply smoothed quantile-based scaling to better distribute scores
    percentiles = np.percentile(normalized_scores, [0, 10, 25, 50, 75, 90, 100])
    target_percentiles = [0, 25, 40, 60, 80, 90, 100]  # Desired distribution
    
    # Map percentiles using piecewise linear interpolation
    final_scores = np.zeros_like(normalized_scores)
    for i in range(len(percentiles) - 1):
        mask = (normalized_scores >= percentiles[i]) & (normalized_scores <= percentiles[i+1])
        if np.any(mask):
            final_scores[mask] = np.interp(
                normalized_scores[mask], 
                [percentiles[i], percentiles[i+1]], 
                [target_percentiles[i], target_percentiles[i+1]]
            )
    
    return final_scores, feature_names, features, pca, scaler

In [ ]:
# Score interpretations
def apply_ml_scoring(df):
    """
    Apply  ML-based scoring with improved interpretability
    and return annotated dataframe with feature importance.
    """
    # Make a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()
    
    # Apply ML scoring with  analytics
    scores, feature_names, features, pca, scaler = ml_sle_variant_scoring(df_copy)
    df_copy['ml_score'] = scores
    
    #  Calculate feature importance using PCA loadings
    pca_loadings = pca.components_
    feature_importance = np.abs(pca_loadings[0, :])  # Use first principal component
    importance_dict = dict(zip(feature_names, feature_importance))
    
    # Sort features by importance
    sorted_features = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)
    print("Top contributing features to variant scoring:")
    for feature, importance in sorted_features[:5]:
        print(f"  - {feature}: {importance:.3f}")
    
    #  Add variant score explanations
    explanations = []
    for i, score in enumerate(scores):
        # Get top contributing features for this variant
        variant_features = features[i]
        scaled_features = scaler.transform([variant_features])[0]
        
        # Calculate feature contributions
        contributions = {}
        for j, feature in enumerate(feature_names):
            contributions[feature] = scaled_features[j] * feature_importance[j]
        
        # Sort contributions
        sorted_contribs = sorted(contributions.items(), key=lambda x: abs(x[1]), reverse=True)
        top_factors = [f"{name} ({contrib:.2f})" for name, contrib in sorted_contribs[:3]]
        
        # Generate explanation
        if score >= 85:
            explanation = f"Very high risk variant. Top factors: {', '.join(top_factors)}"
        elif score >= 70:
            explanation = f"High risk variant. Top factors: {', '.join(top_factors)}"
        elif score >= 55:
            explanation = f"Moderate-high risk variant. Top factors: {', '.join(top_factors)}"
        elif score >= 40:
            explanation = f"Moderate risk variant. Top factors: {', '.join(top_factors)}"
        elif score >= 25:
            explanation = f"Low-moderate risk variant. Notable factors: {', '.join(top_factors)}"
        elif score >= 10:
            explanation = f"Low risk variant."
        else:
            explanation = f"Very low risk variant."
            
        explanations.append(explanation)
    
    #  Create risk interpretations with more context
    interpretations = []
    for score in scores:
        if score >= 85:
            interpretations.append("Very high risk SLE-associated variant")
        elif score >= 70:
            interpretations.append("High risk SLE-associated variant")
        elif score >= 55:
            interpretations.append("Moderate-high risk SLE-associated variant")
        elif score >= 40:
            interpretations.append("Moderate risk SLE-associated variant")
        elif score >= 25:
            interpretations.append("Low-moderate risk SLE-associated variant")
        elif score >= 10:
            interpretations.append("Low risk SLE-associated variant")
        else:
            interpretations.append("Very low risk SLE-associated variant")
    
    # Add explanations and interpretations to dataframe
    df_copy['ml_interpretation'] = interpretations
    df_copy['ml_explanation'] = explanations
    
    # Sort by score
    df_sorted = df_copy.sort_values('ml_score', ascending=False)
    
    # Create a more detailed distribution analysis
    print("\nML Score Distribution:")
    categories = [
        ("Very high risk (85-100)", lambda s: s >= 85),
        ("High risk (70-84)", lambda s: 70 <= s < 85),
        ("Moderate-high risk (55-69)", lambda s: 55 <= s < 70),
        ("Moderate risk (40-54)", lambda s: 40 <= s < 55),
        ("Low-moderate risk (25-39)", lambda s: 25 <= s < 40),
        ("Low risk (10-24)", lambda s: 10 <= s < 25),
        ("Very low risk (0-9)", lambda s: s < 10)
    ]
    
    for category_name, condition in categories:
        count = sum(1 for s in scores if condition(s))
        print(f"{category_name}: {count} variants ({count/len(scores)*100:.1f}%)")
    
    return df_sorted


In [ ]:
# dealing the missing data
def identify_high_risk_variants_with_missing_data(df_scored):
    """
    Specifically identify high-risk variants that are missing prediction scores.
    These variants might be overlooked by traditional filtering approaches.
    """
    # Check which prediction columns exist in the dataset
    prediction_columns = ['PolyPhen', 'SIFT', 'CADD_PHRED', 'ClinPred']
    available_columns = [col for col in prediction_columns if col in df_scored.columns]
    
    if not available_columns:
        print("No prediction score columns found in dataset.")
        return df_scored
    
    # Create a missing data flag
    df_scored['missing_prediction_data'] = False
    df_scored['missing_predictions_count'] = 0
    
    # Count missing prediction data for each variant
    for col in available_columns:
        df_scored['missing_prediction_data'] |= pd.isna(df_scored[col])
        df_scored['missing_predictions_count'] += pd.isna(df_scored[col]).astype(int)
    
    # Calculate the percentage of missing prediction data
    df_scored['missing_predictions_percent'] = (df_scored['missing_predictions_count'] / len(available_columns)) * 100
    
    # Add a score adjustment for variants with missing data but other strong indicators
    # This helps identify potential high-risk variants that would be missed by standard filters
    df_scored['missing_data_risk_adjustment'] = 0.0
    
    for idx, row in df_scored.iterrows():
        if row['missing_predictions_count'] > 0:
            adjustment = 0.0
            adjustment_reasons = []
            
            # Check for strong disease association
            if 'SLE' in df_scored.columns and 'Healthy' in df_scored.columns:
                sle_presence = float(row['SLE']) if pd.notna(row['SLE']) else 0
                healthy_presence = float(row['Healthy']) if pd.notna(row['Healthy']) else 0
                
                if sle_presence > 0 and healthy_presence == 0:
                    adjustment += 15.0  # SLE-specific variants
                    adjustment_reasons.append("SLE-specific")
                elif sle_presence > healthy_presence * 2:
                    adjustment += 10.0  # SLE-enriched variants
                    adjustment_reasons.append("SLE-enriched")
            
            # Check for important structural consequences
            if 'Consequence' in df_scored.columns and pd.notna(row['Consequence']):
                consequences = str(row['Consequence']).split(',')
                max_weight = max(weights.get(consequence.strip(), 0) for consequence in consequences)
                
                if max_weight >= 15:  # High-impact consequences
                    adjustment += 12.0
                    adjustment_reasons.append("High-impact consequence")
                elif max_weight >= 10:  # Moderate-impact consequences
                    adjustment += 8.0
                    adjustment_reasons.append("Moderate-impact consequence")
            
            # Check for amino acid changes
            if 'Amino_acids' in df_scored.columns and pd.notna(row['Amino_acids']):
                aa_str = str(row['Amino_acids'])
                if (aa_str != "0" and aa_str.strip() != "" and not aa_str.endswith("=") and aa_str != "-"):
                    adjustment += 10.0
                    adjustment_reasons.append("Amino acid change")
                    
                    # Check for significant property changes
                    if '/' in aa_str:
                        aa_parts = aa_str.split('/')
                        if len(aa_parts) == 2:
                            # Define amino acid property groups
                            hydrophobic = set(['A', 'I', 'L', 'M', 'F', 'V', 'P', 'G'])
                            polar = set(['Q', 'N', 'H', 'S', 'T', 'Y', 'C', 'W'])
                            charged_positive = set(['K', 'R'])
                            charged_negative = set(['D', 'E'])
                            
                            def get_aa_group(aa):
                                if aa in hydrophobic: return 'hydrophobic'
                                if aa in polar: return 'polar'
                                if aa in charged_positive: return 'positive'
                                if aa in charged_negative: return 'negative'
                                return 'unknown'
                            
                            orig = aa_parts[0][0] if len(aa_parts[0]) > 0 else ''
                            new = aa_parts[1][0] if len(aa_parts[1]) > 0 else ''
                            
                            if orig and new and orig != new:
                                orig_group = get_aa_group(orig)
                                new_group = get_aa_group(new)
                                
                                # Changes between property groups are more severe
                                if orig_group != new_group and orig_group != 'unknown' and new_group != 'unknown':
                                    adjustment += 5.0
                                    adjustment_reasons.append("Property-changing AA substitution")
            
            # Check for rarity - rare variants with missing data might be important
            if 'AF' in df_scored.columns:
                af = float(row['AF']) if pd.notna(row['AF']) else 0
                if af == 0:
                    adjustment += 8.0  # Unique variants
                    adjustment_reasons.append("Unique variant")
                elif af < 0.001:
                    adjustment += 5.0  # Very rare variants
                    adjustment_reasons.append("Very rare variant")
            
            # Check for high impact
            if 'IMPACT' in df_scored.columns and pd.notna(row['IMPACT']):
                if row['IMPACT'] == 'HIGH':
                    adjustment += 15.0
                    adjustment_reasons.append("HIGH impact")
                elif row['IMPACT'] == 'MODERATE':
                    adjustment += 8.0
                    adjustment_reasons.append("MODERATE impact")
            
            # Apply the adjustment, scaled by how much data is missing
            # More missing data = higher potential adjustment
            missing_ratio = row['missing_predictions_count'] / len(available_columns)
            final_adjustment = adjustment * missing_ratio
            
            # Only apply significant adjustments
            if final_adjustment >= 5.0:
                df_scored.at[idx, 'missing_data_risk_adjustment'] = final_adjustment
                
                # Also create an explanation field if it doesn't exist
                if 'missing_data_explanation' not in df_scored.columns:
                    df_scored['missing_data_explanation'] = ""
                
                # Add explanation
                df_scored.at[idx, 'missing_data_explanation'] = f"Missing data risk factors: {', '.join(adjustment_reasons)}"
    
    # Apply the adjustment to create a new adjusted score
    # but only for variants with missing data
    df_scored['ml_score_adjusted'] = df_scored['ml_score']
    mask = df_scored['missing_predictions_count'] > 0
    df_scored.loc[mask, 'ml_score_adjusted'] = df_scored.loc[mask, 'ml_score'] + df_scored.loc[mask, 'missing_data_risk_adjustment']
    
    # Cap the adjusted score at 100
    df_scored['ml_score_adjusted'] = df_scored['ml_score_adjusted'].clip(upper=100)
    
    # Create new interpretation based on adjusted score
    df_scored['ml_interpretation_adjusted'] = df_scored['ml_interpretation']
    
    # Update interpretations for adjusted scores
    for idx, row in df_scored.iterrows():
        if row['missing_predictions_count'] > 0 and row['missing_data_risk_adjustment'] > 0:
            adjusted_score = row['ml_score_adjusted']
            
            if adjusted_score >= 85:
                df_scored.at[idx, 'ml_interpretation_adjusted'] = "Very high risk SLE-associated variant (adjusted for missing data)"
            elif adjusted_score >= 70:
                df_scored.at[idx, 'ml_interpretation_adjusted'] = "High risk SLE-associated variant (adjusted for missing data)"
            elif adjusted_score >= 55:
                df_scored.at[idx, 'ml_interpretation_adjusted'] = "Moderate-high risk SLE-associated variant (adjusted for missing data)"
            elif adjusted_score >= 40:
                df_scored.at[idx, 'ml_interpretation_adjusted'] = "Moderate risk SLE-associated variant (adjusted for missing data)"
    
    # Identify potentially high-risk variants with missing data
    high_risk_missing = df_scored[
        (df_scored['missing_predictions_count'] > 0) & 
        (df_scored['ml_score_adjusted'] >= 40)  # Moderate or higher adjusted risk score
    ].copy()
    
    # Sort by adjusted risk score
    high_risk_missing = high_risk_missing.sort_values('ml_score_adjusted', ascending=False)
    
    # Create a flag for variants that were promoted to higher risk categories
    df_scored['risk_category_promoted'] = False
    
    for idx, row in df_scored.iterrows():
        if row['missing_predictions_count'] > 0:
            original_score = row['ml_score']
            adjusted_score = row['ml_score_adjusted']
            
            # Check if variant moved to a higher risk category
            if (original_score < 85 and adjusted_score >= 85) or \
               (original_score < 70 and adjusted_score >= 70) or \
               (original_score < 55 and adjusted_score >= 55) or \
               (original_score < 40 and adjusted_score >= 40):
                df_scored.at[idx, 'risk_category_promoted'] = True
    
    # Identify newly discovered high-risk variants
    promoted_variants = df_scored[df_scored['risk_category_promoted'] == True].sort_values('ml_score_adjusted', ascending=False)
    
    return df_scored

In [ ]:

# Step 1: Apply the ML scoring
ml_scored_variants = apply_ml_scoring(result_df)
updated_scored_variants = Final_table_unfiltered.copy()

# Step 2: Identify variants with missing prediction data that may still be high-risk
variants_with_adjusted_scores = identify_high_risk_variants_with_missing_data(ml_scored_variants)
updated_scored_variants = updated_scored_variants.merge(
    variants_with_adjusted_scores[['POS', 'ml_score_adjusted', 'ml_interpretation']], 
    on='POS'
)

In [7]:
updated_scored_variants = updated_scored_variants.rename(columns={'ml_score_adjusted': 'ml_score'})

In [8]:
#scored_variants = scored_variants.merge(Final_table_unfiltered[['POS', 'score']], on='POS')
#cols = list(scored_variants.columns)
#cols.insert(len(cols)-2, cols.pop(cols.index('score')))
#scored_variants = scored_variants[cols]
updated_scored_variants = updated_scored_variants.rename(columns={'score': 'manual_ranking_score'})


In [10]:
def combine_manual_and_ml_scores(df, ml_weight):
    """
    Combine manual ranking score and ML score into a final score
    
    Parameters:
    df: DataFrame with both 'ml_score' and 'manual_ranking_score' columns
       (manual_ranking_score is expected to be on a 0-50 scale)
    ml_weight: Weight for ML score (between 0 and 1), manual weight will be (1-ml_weight)
    
    Returns:
    DataFrame with additional 'final_score' and 'final_score_interpretation' columns
    """
    # Ensure both scores exist
    if 'ml_score' not in df.columns or 'manual_ranking_score' not in df.columns:
        raise ValueError("DataFrame must contain both 'ml_score' and 'manual_ranking_score' columns")
    
    # Create a copy of the dataframe
    df_copy = df.copy()
    
    # Normalize manual score from 0-50 to 0-100 scale
    df_copy['manual_ranking_score_normalized'] = df_copy['manual_ranking_score'] 
    
    # Combine scores using weighted average
    manual_weight = 1 - ml_weight
    df_copy['final_score'] = (
        df_copy['ml_score'] * ml_weight + 
        df_copy['manual_ranking_score_normalized'] * manual_weight
    )
    
    # Create interpretations based on final score
    interpretations = []
    for score in df_copy['final_score']:
        if score >= 85:
            interpretations.append("Very high risk")
        elif score >= 70:
            interpretations.append("High risk")
        elif score >= 55:
            interpretations.append("Moderate-high risk")
        elif score >= 40:
            interpretations.append("Moderate risk")
        elif score >= 25:
            interpretations.append("Low-moderate risk")
        elif score >= 10:
            interpretations.append("Low risk")
        else:
            interpretations.append("Very low risk")
    
    df_copy['final_score_interpretation'] = interpretations
    
    # Create a distribution analysis of the final scores
    score_distribution = {
        "very_high": sum(1 for s in df_copy['final_score'] if s >= 85),
        "high": sum(1 for s in df_copy['final_score'] if 70 <= s < 85),
        "moderate_high": sum(1 for s in df_copy['final_score'] if 55 <= s < 70),
        "moderate": sum(1 for s in df_copy['final_score'] if 40 <= s < 55),
        "low_moderate": sum(1 for s in df_copy['final_score'] if 25 <= s < 40),
        "low": sum(1 for s in df_copy['final_score'] if 10 <= s < 25),
        "very_low": sum(1 for s in df_copy['final_score'] if s < 10)
    }
    
    # Print score distribution
    print("Final Combined Score Distribution:")
    for category, count in score_distribution.items():
        print(f"{category.replace('_', ' ').title()}: {count} variants ({count/len(df_copy)*100:.1f}%)")
    
    # Remove intermediate columns that are no longer needed
    df_copy = df_copy.drop(["manual_ranking_score_normalized"], axis=1)
    
    return df_copy


#
# Then combine with manual scores:
final_variants = combine_manual_and_ml_scores(updated_scored_variants, ml_weight=0.5)
#
# To drop additional columns if needed:
final_variants = final_variants.drop(["ml_interpretation"], axis=1)

Final Combined Score Distribution:
Very High: 2 variants (0.3%)
High: 32 variants (4.8%)
Moderate High: 166 variants (25.0%)
Moderate: 130 variants (19.6%)
Low Moderate: 247 variants (37.3%)
Low: 67 variants (10.1%)
Very Low: 19 variants (2.9%)


In [11]:
# Sort in descending order (highest score first)
df_final_sorted = final_variants.sort_values('final_score', ascending=False)
df_final_sorted

,POS,Datasets,Healthy,SLE,Occurrence,REF,ALT,GQ,QUAL,DP,...,AF,Protein_position,Amino_acids,CADD_PHRED,SIFT,PolyPhen,manual_ranking_score,ml_score,final_score,final_score_interpretation
0,68003029,2,0,3,3,G,T,99.0,875.306667,65.0,...,0.0028,129,L129I,26.000,0.02,0.951,85.855719,100.000000,92.927860,Very high risk
1,67999234,2,0,3,3,G,A,99.0,1490.640000,72.0,...,0.0028,209,P209L,25.400,0.00,0.996,71.711330,100.000000,85.855665,Very high risk
2,67991568,2,0,1,1,C,T,99.0,968.640000,41.0,...,0.0024,591,G591E,17.490,0.48,0.995,64.518414,100.000000,82.259207,High risk
3,67999680,2,0,1,1,C,A,50.0,42.640000,50.0,...,0.0000,131,R131S,25.000,0.00,0.350,57.554599,100.000000,78.777300,High risk
18,67991757,3,0,1,1,C,T,99.0,0.000000,569.0,...,0.0000,528,R528H,29.200,0.04,0.891,51.914286,100.000000,75.957143,High risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,67991207,3,1,0,1,C,G,99.0,0.000000,178.0,...,0.0000,-,-,2.146,0.00,0.000,13.728571,0.335469,7.032020,Very low risk
623,67997644,3,1,0,1,C,A,99.0,0.000000,60.0,...,0.0000,-,-,2.161,0.00,0.000,12.030874,1.733237,6.882056,Very low risk
596,67991322,3,1,0,1,C,G,99.0,0.000000,308.0,...,0.0000,-,-,1.459,0.00,0.000,13.728571,0.000000,6.864286,Very low risk
622,67996064,3,1,0,1,G,T,99.0,0.000000,75.0,...,0.0000,-,-,2.950,0.00,0.000,12.031408,0.013252,6.022330,Very low risk


In [36]:
pd.set_option('display.max_columns', None)

In [12]:
df_final_sorted.to_csv("Combined_ranking_final_table_unfiltered_the_FINAL.csv", index = False)